In [30]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

from numpy import array


"""from keras.preprocessing.text import Tokenizer

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding"""


'from keras.preprocessing.text import Tokenizer\n\nfrom keras.preprocessing.text import one_hot\nfrom keras.preprocessing.sequence import pad_sequences\nfrom keras.models import Sequential\nfrom keras.layers.core import Activation, Dropout, Dense\nfrom keras.layers import Flatten\nfrom keras.layers import GlobalMaxPooling1D\nfrom keras.layers.embeddings import Embedding'

In [31]:
reviews = pd.read_csv("~/Desktop/ece684/movies.csv")
reviews


,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0
...,...,...
4995,This is the kind of picture John Lassiter woul...,1
4996,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,NBC should be ashamed. I wouldn't allow my chi...,0
4998,This movie is a clumsy mishmash of various gho...,0


In [32]:
reviews.isnull().values.any()
# The data set has no missing values



False

In [33]:
reviews["text"][6]


"I saw a screening of this movie last night. I had high expectations going into it, but was definitely disappointed. Within 5 minutes of the opening, Williams is already campaigning for his presidency. And he becomes president in the first 40 minutes. So there goes all that aspect of the movie. The first half hour are hilarious. Don't get me wrong, the movie has its moments. But after the first half hour, it takes a turn for the worst. It becomes less of a comedy, and more of a thriller/drama/love story...which is pointless. the movie goes nowhere and stands still for a good 30 minutes. there are laughs interspersed here and there, but the consistently funny part is in the beginning and only the beginning. at one point, the biggest cheer i heard in the audience is when a person in the crowd yelled 'boooo' during a very confusingly emotional scene. Williams gives a great performance, right on par with his comedic style. Walken also delivers a strong supporting role as only he can. I thi

In [34]:
import re

# Reformats Reviews


def cleanReview(rev):
    # Removing anything within a HTML tag
    edited_rev = re.compile(r"<[^>]+>").sub(" ", rev)
    # Removing Punctuation
    edited_rev = re.sub(r"[^\w\s]", " ", edited_rev)
    # Removing Numbers
    edited_rev = re.sub(r"[0-9]", " ", edited_rev)
    # Removing single characters
    edited_rev = re.sub(r"\s+[a-zA-Z]\s+", " ", edited_rev)
    # Removing multiple spaces
    edited_rev = re.sub(r"\s+", " ", edited_rev)

    return edited_rev


In [35]:
# Checking Cleaning Function
"""rev1 = reviews["text"][3]
rev= cleanReview(rev1)
rev"""



'rev1 = reviews["text"][3]\nrev= cleanReview(rev1)\nrev'

In [36]:
review_list = []

# Adding all of the movie reviews to a list
for r in reviews["text"]:
    review_list.append(cleanReview(r))


In [37]:
#Start of tokenizing
words = " ".join(review_list)
words = list(words.split(" "))

In [38]:
#Getting word counts
totalCounts = {}
for j in range(len(words)):
    if words[j] not in totalCounts:
        totalCounts[words[j]] = 1
    if words[j] in totalCounts:
        totalCounts[words[j]] += 1

In [39]:
#Sorting dictionary to have highest word counts first
sorted_counts=dict(sorted(totalCounts.items(), key=lambda x: x[1], reverse=True))

In [40]:
#Making the highest word count value be 1, etc.
w_to_i = {w:i+1 for i, (w) in enumerate(sorted_counts)}

In [41]:
# Encoding the words by replacing the word with its given integer
X = []
for m in review_list:
    r = [w_to_i[w] for w in m.split()]
    X.append(r)

In [42]:
import statistics
lengths = []
for i in range(len(X)):
    l = len(X[i])
    lengths.append(l)
statistics.mean(lengths)

221.0492

In [43]:
#Padding for different review lengths
X_array = np.zeros((len(X), 220), dtype =int)
for i, rev2 in enumerate(X):
    rev_length = len(rev2)

    if rev_length <= 220:
        zeros = list(np.zeros(220-rev_length))
        mat = zeros + rev2
    elif rev_length > 220:
        mat = rev2[0:220]
    X_array[i,:] = np.array(mat)

In [44]:
# Creating a training and test data sets
y = reviews["label"]

y = np.array(list(map(lambda x: 1 if x == 1 else 0, y)))

In [45]:
X_train, X_test, y_train, y_test = train_test_split(
    X_array, y, test_size=0.20, random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42
)

In [46]:
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn

# create Tensor Dataset
train_data = TensorDataset(torch.LongTensor(X_train), torch.LongTensor(y_train))
valid_data=TensorDataset(torch.LongTensor(X_valid), torch.LongTensor(y_valid))
test_data = TensorDataset(torch.LongTensor(X_test), torch.LongTensor(y_test))


In [47]:
#dataloader
batch_size=50
train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader=DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader=DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [48]:
class SentimentalLSTM(nn.Module):
    def __init__(
        self, num_words, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5
    ):
        super().__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # Embedding and LSTM layers
        self.embedding = nn.Embedding(num_words, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True
        )

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # Linear and sigmoid layer
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size()

        #   Embadding and LSTM output
        embedd = self.embedding(x)
        lstm_out, hidden = self.lstm(embedd, hidden)

        # stack up the lstm output
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layers
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.fc3(out)
        sig_out = self.sigmoid(out)

        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]

        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden


In [49]:
# Instantiate the model w/ hyperparams
#vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
num_words = len(words)
output_size = 1
embedding_dim = 220
hidden_dim = 256
n_layers = 2

net = SentimentalLSTM(num_words, output_size, embedding_dim, hidden_dim, n_layers)
print(net)

SentimentalLSTM(
  (embedding): Embedding(1110252, 220)
  (lstm): LSTM(220, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [50]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

# training params

epochs = 3 # is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()


net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)


    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs=inputs.cuda()
            labels=labels.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()
        
        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                inputs, labels = inputs, labels 
                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))


Epoch: 2/3... Step: 100... Loss: 0.644037... Val Loss: 0.659365


In [52]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()


    output, h = net(inputs, h)

    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.712
Test accuracy: 0.648
